In [1]:
#### Install Libs

!pip install ccxt
!pip install schedule
## TA - LIB ##
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

     |████████████████████████████████| 2.1MB 7.0MB/s 
     |████████████████████████████████| 163kB 33.2MB/s 
     |████████████████████████████████| 3.2MB 34.0MB/s 
     |████████████████████████████████| 1.3MB 35.9MB/s 
     |████████████████████████████████| 235kB 30.1MB/s 
     |████████████████████████████████| 143kB 38.0MB/s 
  Created wheel for yarl: filename=yarl-1.1.0-cp37-cp37m-linux_x86_64.whl size=241443 sha256=f05cc257e4ad0bf17da2ccce8318417a64c2dc68d2fd08f201dc49ed7907c1b8
  Stored in directory: /root/.cache/pip/wheels/96/26/b9/7972042a1a69556ba7f38b8a301dc3a9ddac1b8f61f02fbfd8
Successfully built yarl
Selecting previously unselected package libta-lib0.
(Reading database ... 160980 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setti

In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import seaborn as sns
import math
import ccxt
import matplotlib.pyplot as plt
%matplotlib inline
import talib as ta
import requests
import schedule
import warnings


pd.options.display.max_columns = None
warnings.filterwarnings('ignore')
# sns.set_style('darkgrid')



In [5]:
######### SET UP EXCHANGE ########
apiKey=  ''
secret = ''
sub_account = ''

exchange  = ccxt.ftx({
    'verbose': False,
    'apiKey': apiKey,
    'secret': secret,
    'enableRateLimit': True,
    'headers': {
        'FTX-SUBACCOUNT': sub_account,
    },
})


# mpf.available_styles()
### SETTING ###
symbol= "UNI-PERP"
timeframe ='1d'
###########


In [15]:
def get_data(symbol):
  df= pd.DataFrame(exchange.fetch_ohlcv(symbol,timeframe),columns=['date','open','high','low','close','volume'])
  # df.drop(columns='volume',inplace=True)
  df['volume'] =(df['volume'].astype('int'))
  df['date']=df['date'].apply(exchange.iso8601)
  df['date']=pd.to_datetime(df['date'])
#   df['date']=df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M'))
#   df['date']=pd.to_datetime(df['date'])
  df.set_index('date',inplace=True)

#   df.dropna(inplace=True)
  return df



def anlysis(dataframe):
    try:
        df =dataframe
        up, mid, low = ta.BBANDS(df['close'], timeperiod=50, nbdevup=2, nbdevdn=2, matype=0)
        #   df['chg'] =df.close - df.close.shift(1)
        df['atr'] = ta.ATR(df['high'],df['low'],df['close'],timeperiod= 10)
        df['rsi']= ta.RSI(df['close'],timeperiod=10)
        df.dropna(inplace=True)
#         print(df.tail(5))
        return df
    except Exception as e:
        print(str(e))
        
def sendtext(message):
    try:    
        token = '' ### ใส่ Token ตรงนี้ครับผม
        url = 'https://notify-api.line.me/api/notify'
        header = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer '+token}   
        command = {'message':message}
        return requests.post(url, headers=header, data = command)
    except Exception as e:
        print(e)
        

In [29]:
def AlertPrice(trigger,symbol) :### Trigger x.x % 
    df = get_data(symbol)
    df= anlysis(df)
    now_price = df['close'][-1]
    rsi = df['rsi'][-1]
    atr = df['atr'][-1]
    ref_price = df['close'][-2]
    diff = (now_price - ref_price) /ref_price  *100
    diff = round(diff,3)



    if abs(diff) >= trigger :
        time_stamp = dt.datetime.now().strftime('%D %H:%M:%S')
        msg = f' Alert :{time_stamp} \n•{symbol} \n•Price {ref_price}:>>>{now_price}>>{diff}% \n•RSI : {rsi:.2f} ATR : {(atr/now_price*100):.2f}%'
        print('#'*60,'\n')
        print(msg)
        print('#'*60,'\n')

        sendtext(msg)
        

    else:
        time_stamp = dt.datetime.now().strftime('%D %H:%M:%S')
        msg = f' Check Price :{time_stamp} \n•{symbol} \n•Price {ref_price}:>>>{now_price}>>{diff}% \n•RSI : {rsi:.2f} ATR : {(atr/now_price*100):.2f}%'
        print('#'*60,'\n')
        print(msg)
        print('#'*60,'\n')




In [ ]:

symbols = 'UNI-PERP'
timeframe = '15m'

while True:
    AlertPrice(1.0,symbols)
    time.sleep(300)

############################################################ 

 Check Price :03/23/21 16:54:29 
•UNI-PERP 
•Price 33.959:>>>33.736>>-0.657% 
•RSI : 58.80 ATR : 1.12%
############################################################ 



In [31]:
############ Multi Symbols ##########

symbols = ['UNI-PERP','ETH-PERP','BTC-PERP']
timeframe = '1h'
while True:

  for i in symbols:
    if i == 'UNI-PERP':
      AlertPrice(2.0,i)
    else : 
        AlertPrice(1.5,i)
  time.sleep(300)


############################################################ 

 Check Price :03/23/21 16:35:16 
•UNI-PERP 
•Price 33.231:>>>33.789>>1.679% 
•RSI : 53.39 ATR : 2.45%
############################################################ 

############################################################ 

 Check Price :03/23/21 16:35:16 
•ETH-PERP 
•Price 1704.5:>>>1716.5>>0.704% 
•RSI : 56.20 ATR : 1.24%
############################################################ 

############################################################ 

 Check Price :03/23/21 16:35:17 
•BTC-PERP 
•Price 55209.0:>>>55672.0>>0.839% 
•RSI : 60.12 ATR : 1.30%
############################################################ 



KeyboardInterrupt: ignored